In [1]:
# Initiate DB
from src.db_models.base_model import db
from src.db_models.sample import Sample
from src.db_models.ml_model import MlModel
from src.db_models.prediction import Prediction
from src.db_models.test_sample import TestSample

# Connect to DB
db.connect()
# Migrate tables
db.create_tables([MlModel, TestSample, Sample, Prediction])


In [2]:
# Get data
import pandas as pd

data = pd.read_csv("../data/Boiler_emulator_dataset_new.csv")

predictions = data["Class"]
features = data.drop(columns=['Condition', 'Class'])

In [3]:
# Create Primary model
model_file = "models/keras_model_2.keras"

MlModel.update(is_active=False).execute()
model_record = MlModel.create(model_path=model_file, is_active=True)

In [4]:
# Fill DB with data

from sklearn.model_selection import train_test_split

X_train_full , X_test , y_train_full , y_test = train_test_split(features, predictions, random_state=42)

# Insert to DB
for index, row in X_test.iterrows():
    sample = TestSample.create(
        fuel_mdot = row["Fuel_Mdot"],
        tair = row["Tair"],
        treturn = row["Treturn"],
        tsupply = row["Tsupply"],
        water_mdot = row["Water_Mdot"],
        prediction = y_test[index])
        
X_test

,Fuel_Mdot,Tair,Treturn,Tsupply,Water_Mdot
8834,1,283,333.0,343.920469,4.5
626,1,289,333.0,351.416977,4.0
20841,2,293,333.0,345.947128,5.0
5549,3,293,333.0,339.742323,10.0
13218,3,283,333.0,342.588395,3.5
...,...,...,...,...,...
18387,1,289,333.0,335.032964,12.0
20472,2,289,333.0,340.805189,6.5
410,1,287,333.0,359.279102,3.5
21362,2,297,333.0,337.628698,11.0


In [5]:
db.close()

True